# Pair ETL I

### **Ejercicios ETL Parte I**

En este caso trabajas en una empresa de venta al por menor de productos italianos y debes realizar la limpieza, transformación e integración de datos de ventas, productos y clientes para su análisis.

Los pasos que deberás seguir en este ejercicio son:

1. **Lectura de la Información:**
   * Leer los archivos CSV (`ventas.csv`, `productos.csv`, `clientes.csv`).
   * Explorar los conjuntos de datos para comprender su estructura, columnas, tipos de datos, etc.

In [80]:
# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

# ----------------------------------------------------------------------
from datetime import date, datetime
from dotenv import load_dotenv
import os
import requests

In [81]:
df_clientes = pd.read_csv(r'archivos\clientes.csv')
df_ventas = pd.read_csv(r'archivos\ventas.csv')

In [82]:
# Paso 1: Leer el archivo como texto
with open('archivos/productos.csv', "r", encoding="utf-8") as f:
    lines = f.readlines()

# Paso 2: Procesar líneas y dividir columnas
data = []
for line in lines[1:]:  # Saltar encabezado
    parts = line.strip().split(",")
    primeras_5 = parts[:5]  # ID, Nombre_Producto, Categoría, Precio, Origen
    descripcion = ",".join(parts[5:])  # Unir todo lo que sigue como descripción
    data.append(primeras_5 + [descripcion])

# Paso 3: Crear DataFrame
columnas = ["ID", "Nombre_Producto", "Categoría", "Precio", "Origen", "Descripción"]
df_products = pd.DataFrame(data, columns=columnas)

# Paso 4: (opcional) convertir Precio a float
df_products["Precio"] = df_products["Precio"].astype(float)

In [83]:
df_products.head(2)

,ID,Nombre_Producto,Categoría,Precio,Origen,Descripción
0,A1,Pizza Margherita,Platos Preparados,8.99,Italia,"Clásica pizza italiana con tomate, mozzarella ..."
1,B2,Risotto de Champiñones,Platos Preparados,6.75,Italia,"Risotto cremoso con champiñones frescos, una d..."


In [84]:
df_products.shape

(33, 6)

In [85]:
df_products.describe().T

,count,mean,std,min,25%,50%,75%,max
Precio,33.0,6.563939,2.441529,2.49,4.75,5.99,7.89,12.45


In [86]:
df_products.describe(include='O').T

,count,unique,top,freq
ID,33,33,A1,1
Nombre_Producto,33,33,Pizza Margherita,1
Categoría,33,17,Platos Preparados,5
Origen,33,1,Italia,33
Descripción,33,33,"Clásica pizza italiana con tomate, mozzarella ...",1


In [87]:
df_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               33 non-null     object 
 1   Nombre_Producto  33 non-null     object 
 2   Categoría        33 non-null     object 
 3   Precio           33 non-null     float64
 4   Origen           33 non-null     object 
 5   Descripción      33 non-null     object 
dtypes: float64(1), object(5)
memory usage: 1.7+ KB


In [88]:
df_products[df_products.isna().any(axis=1)]

,ID,Nombre_Producto,Categoría,Precio,Origen,Descripción


In [89]:
df_products.tail(2)

,ID,Nombre_Producto,Categoría,Precio,Origen,Descripción
31,FF32,Cassata Siciliana,Postres,10.99,Italia,"Tarta siciliana con capas de bizcocho, ricotta..."
32,GG33,Ciceri e Tria,Platos Preparados,5.89,Italia,Plato de pasta tradicional de Apulia con garba...


In [90]:
df_clientes.head(2)

,id,first_name,last_name,email,gender,City,Country,Address
0,1,Cheri,Dunsmore,cdunsmore0@instagram.com,Female,Palma De Mallorca,Spain,076 Rockefeller Crossing
1,2,Hunt,Bartomeu,hbartomeu1@nsw.gov.au,Male,Lugo,Spain,0046 Utah Junction


In [93]:
df_clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          1000 non-null   int64 
 1   first_name  1000 non-null   object
 2   last_name   1000 non-null   object
 3   email       973 non-null    object
 4   gender      923 non-null    object
 5   City        876 non-null    object
 6   Country     846 non-null    object
 7   Address     959 non-null    object
dtypes: int64(1), object(7)
memory usage: 62.6+ KB


In [91]:
df_clientes.shape

(1000, 8)

In [94]:
df_ventas.head(2)

,ID_Cliente,ID_Producto,Fecha_Venta,Cantidad,Total
0,723,A1,2023-11-22,2,17.98
1,498,C3,2023-11-21,1,5.49


In [95]:
df_ventas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID_Cliente   100 non-null    int64  
 1   ID_Producto  100 non-null    object 
 2   Fecha_Venta  100 non-null    object 
 3   Cantidad     100 non-null    int64  
 4   Total        100 non-null    float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ KB


2. **Transformación de Datos:**
   * Limpiar los datos: manejar valores nulos, eliminar duplicados si los hay, corregir errores tipográficos, etc.
   * Realizar la integración de datos: unir los conjuntos de datos apropiados para obtener una tabla única que contenga información de ventas junto con detalles de productos y clientes.
   * Aplicar transformaciones relevantes según sea necesario: por ejemplo, convertir tipos de datos, renombrar columnas, crear nuevas características derivadas, etc.

**NOTA** Este ejercicio debe realizarse en un archivo `.py`

In [97]:
df_clientes.head(3)

,id,first_name,last_name,email,gender,City,Country,Address
0,1,Cheri,Dunsmore,cdunsmore0@instagram.com,Female,Palma De Mallorca,Spain,076 Rockefeller Crossing
1,2,Hunt,Bartomeu,hbartomeu1@nsw.gov.au,Male,Lugo,Spain,0046 Utah Junction
2,3,Michaeline,Paynton,mpaynton2@narod.ru,Female,NaN,Spain,0 Corry Crossing


In [98]:
df_clientes.isna().sum()

id              0
first_name      0
last_name       0
email          27
gender         77
City          124
Country       154
Address        41
dtype: int64

In [104]:
df_clientes[df_clientes[['gender']].isna().any(axis=1)]

,id,first_name,last_name,email,gender,City,Country,Address
3,4,Filmer,Eirwin,feirwin3@intel.com,NaN,Leon,Spain,5 American Ash Road
13,14,Elena,Crummey,ecrummeyd@imgur.com,NaN,Valladolid,NaN,76694 Sherman Street
32,33,Adoree,Cutchey,acutcheyw@seesaa.net,NaN,NaN,Spain,917 Marcy Parkway
35,36,Louisa,Padfield,lpadfieldz@diigo.com,NaN,"Coruña, A",NaN,76008 Farmco Drive
38,39,Gnni,Date,gdate12@usgs.gov,NaN,"Palmas De Gran Canaria, Las",Spain,16 Ilene Center
...,...,...,...,...,...,...,...,...
957,958,Cindy,Robardet,crobardetql@usda.gov,NaN,Pontevedra,NaN,4857 Everett Circle
962,963,Victor,Brotherwood,vbrotherwoodqq@ning.com,NaN,Palma De Mallorca,Spain,516 Golf Lane
969,970,Niki,Bonsey,nbonseyqx@deliciousdays.com,NaN,Barcelona,Spain,3045 Goodland Crossing
972,973,Cally,Salvadori,csalvadorir0@sitemeter.com,NaN,Ferrol,Spain,8897 Pennsylvania Lane


In [105]:
df_clientes = df_clientes.fillna({'gender': 'unknown', 'email': 'unknown'})
# df.fillna({'gender': 'unknown', 'email': 'unknown'}, inplace=True)

In [109]:
df_clientes[df_clientes[['Country']].isna().any(axis=1)]

,id,first_name,last_name,email,gender,City,Country,Address
7,8,Aubree,Jirsa,unknown,Female,Santander,NaN,8294 Schurz Court
10,11,Chase,Godilington,cgodilingtona@spiegel.de,Male,NaN,NaN,13 Bluestem Court
11,12,Viola,Maggiore,vmaggioreb@cpanel.net,Female,Malaga,NaN,57 Chinook Hill
13,14,Elena,Crummey,ecrummeyd@imgur.com,unknown,Valladolid,NaN,76694 Sherman Street
31,32,Libby,Relf,lrelfv@nih.gov,Female,Elx/Elche,NaN,0 Stang Hill
...,...,...,...,...,...,...,...,...
957,958,Cindy,Robardet,crobardetql@usda.gov,unknown,Pontevedra,NaN,4857 Everett Circle
978,979,Boniface,Fearnside,bfearnsider6@furl.net,Male,Granada,NaN,4353 Calypso Center
979,980,Kassandra,Sallinger,ksallingerr7@istockphoto.com,Female,Badajoz,NaN,0 Fallview Alley
988,989,Jermain,Klassmann,unknown,unknown,Palma De Mallorca,NaN,8083 Burning Wood Pass
